In [ ]:
import cv2
import numpy as np

# Let's capture from the webcam
cap = cv2.VideoCapture(0)

while True:
    # Read each frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame from BGR to HSV
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the range for red color
    lower_red = np.array([0, 120, 70])
    upper_red = np.array([10, 255, 255])

    # Define the range for blue color
    lower_blue = np.array([100, 150, 0])
    upper_blue = np.array([140, 255, 255])

    lower_green = np.array([40, 40, 40])
    upper_green = np.array([90, 255, 255])
    # Create masks for red and blue colors
    red_mask = cv2.inRange(hsv_frame, lower_red, upper_red)
    blue_mask = cv2.inRange(hsv_frame, lower_blue, upper_blue)
    green_mask = cv2.inRange(hsv_frame, lower_green, upper_green)
    # Find contours on the red mask
    red_contours, _ = cv2.findContours(red_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for contour in red_contours:
        if cv2.contourArea(contour) > 500:  # Filter out small contours
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Red color for red object
            cv2.putText(frame, "Red Object", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Find contours on the blue mask
    blue_contours, _ = cv2.findContours(blue_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for contour in blue_contours:
        if cv2.contourArea(contour) > 500:  # Filter out small contours
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Blue color for blue object
            cv2.putText(frame, "Blue Object", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    green_contours, _ = cv2.findContours(green_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for contour in green_contours:
        if cv2.contourArea(contour) > 500:  # Filter out small contours
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # green color for blue object
            cv2.putText(frame, "Green Object", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Show the current frame
    cv2.imshow("Object Tracking", frame)

    # Exit condition
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

: 

In [1]:
import cv2
# Load pre-trained model (yup, this one's deep!)
net = cv2.dnn.readNetFromCaffe('MobileNetSSD_deploy.prototxt', 'MobileNetSSD_deploy.caffemodel')
# Load your favorite video
cap = cv2.VideoCapture('test.mp4')
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # Prepare the image for detection
    blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()
    # Loop through the detections
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.2:  # If the object seems somewhat real
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * [frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]]
            (startX, startY, endX, endY) = box.astype("int")
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
    cv2.imshow('Object Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [3]:
import cv2

# Create the CSRT tracker (can also use KCF, MIL, etc.)
tracker = cv2.TrackerMIL_create()

# Open the video and read the first frame
cap = cv2.VideoCapture('test.mp4')
ret, frame = cap.read()

# Verify if the video frame is successfully read
if not ret:
    print("Failed to read video")
    cap.release()
    exit()

# Select the region of interest (ROI) for tracking
bbox = cv2.selectROI('Tracking', frame, fromCenter=False, showCrosshair=True)

# Check if the bounding box is valid (not empty)
if bbox == (0, 0, 0, 0):
    print("No valid ROI selected")
    cap.release()
    exit()

# Initialize the tracker with the frame and the bounding box
tracker.init(frame, bbox)

# Track the object in subsequent frames
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Update the tracker for the current frame
    success, bbox = tracker.update(frame)
    if success:
        (x, y, w, h) = [int(v) for v in bbox]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    else:
        cv2.putText(frame, "Tracking failed!", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

    # Display the tracking result
    cv2.imshow('Tracking', frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


KeyboardInterrupt: 

: 

In [ ]:
pip uninstall opencv-python
pip install opencv-contrib-python